Analysis of tests over relational metrics of the form 𝑥/𝑦 .

The idea of the method is as follows:
Instead of pushing “user” CTRs into the test, you can construct another metric and analyze it, but it is guaranteed that if the test on this other one "sees" changes, then there are changes in the original metric.
However, the method itself is very simple.
1. Calculate the total CTR in the control group CTRcontrol=sum(likes)/sum(views)
2. Calculate in both groups metric 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠 = likes - CTRcontrol*views for each user
3. Compare the differences in the groups by the t-test according to the metric 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠
The method is simple, it is guaranteed that with a decent sample size, you can increase the sensitivity of your metric for free (or at least not make it worse).

Aнализ тестов над метриками-отношениями вида  𝑥/𝑦 .

Идея метода заключается в следующем:
Вместо того, чтобы заталкивать в тест «поюзерные» CTR, можно сконструировать другую метрику и анализировать ее, но при этом гарантируется , что если тест на этой другой увидит изменения, значит изменения есть и в метрике исходной 
При этом метод сам по себе очень прост. Что это за метрика такая?  

1. Считаем общий CTR в контрольной группе  𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙=𝑠𝑢𝑚(𝑙𝑖𝑘𝑒𝑠)/𝑠𝑢𝑚(𝑣𝑖𝑒𝑤𝑠) 
2. Посчитаем в обеих группах поюзерную метрику  𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠=𝑙𝑖𝑘𝑒𝑠−𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙∗𝑣𝑖𝑒𝑤𝑠 
3. После чего сравним  t-тестом отличия в группах по метрике 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠  

Метод простой, гарантируется, что при приличном размере выборки  можно бесплатно увеличить чувствительность вашей метрики (или, по крайней мере, не сделать хуже). 

Task
1. Analyze the test between groups 0 and 3 by the metric of linearized likes. Is the difference visible? Has 𝑝−𝑣𝑎𝑙𝑢𝑒 gotten smaller?
2. Analyze the test between groups 1 and 2 by the metric of linearized likes. Is the difference visible? Has 𝑝−𝑣𝑎𝑙𝑢𝑒 gotten smaller?


Задача
1. Проанализировать тест между группами 0 и 3 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?
2. Проанализировать тест между группами 1 и 2 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?

In [1]:
# Download libraries.  Запрос библиотек

import pandahouse
import pandas as pd
import seaborn as sns
import numpy as np

from scipy import stats

In [2]:
# Create a connection. Создание соединения

connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator'
}


In [4]:
# Data request (users, their views and likes from groups 0 and 3
# during the time interval from 01/01/23 to 01/07/23 inclusive).
# Запрос данных (пользователи, их просмотры и лайки из групп 0 и 3
#  во временной интервал с 01.01.23 по 07.01.23 включительно)

q = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM simulator_20230120.feed_actions 
WHERE toDate(time) between '2023-01-01' and '2023-01-07'
    and exp_group in (0,3)
GROUP BY exp_group, user_id
"""

df_03 = pandahouse.read_clickhouse(q, connection=connection)

In [5]:
#T-test on groups 0(control) and 3(test)

t_test = stats.ttest_ind(df_03[df_03.exp_group == 0].ctr,
                df_03[df_03.exp_group == 3].ctr,
                equal_var=False)
print(f' p-value = {t_test.pvalue}')

 p-value = 1.055849414662529e-43


In [6]:
#Calculate global CTR

global_CTR_03 = sum(df_03[df_03.exp_group == 0].likes)/sum(df_03[df_03.exp_group == 0].views)

In [7]:
#Calculate linearized 'likes'

df_03['linearized_likes'] = df_03.apply(
    lambda x: x['likes'] - x['views']*global_CTR_03, axis=1)

In [8]:
df_03

,exp_group,user_id,likes,views,ctr,linearized_likes
0,3,115383,9,30,0.300000,2.752916
1,3,123580,13,48,0.270833,3.004666
2,0,4944,8,41,0.195122,-0.537681
3,0,4504,5,15,0.333333,1.876458
4,0,121508,18,88,0.204545,-0.324779
...,...,...,...,...,...,...
20043,3,13781,10,27,0.370370,4.377625
20044,3,130651,18,98,0.183673,-2.407140
20045,0,3681,7,55,0.127273,-4.452987
20046,3,122008,5,41,0.121951,-3.537681


In [9]:
#T-test on linearized data

t_test_03 = stats.ttest_ind(df_03[df_03.exp_group == 0].linearized_likes,
                df_03[df_03.exp_group == 3].linearized_likes,
                equal_var=False)
print(f' p-value = {t_test_03.pvalue}')

 p-value = 5.491424947968532e-52


Both t-tests showed a significant difference.
p-value in linearized test is less.

Оба т-теста показали значимое отличие.          
p-value в линеарезированном тесте меньше

--------------

In [10]:
# Data request (users, their views and likes from groups 1 and 2
# during the time interval from 01/01/23 to 01/07/23 inclusive).
# Запрос данных (пользователи, их просмотры и лайки из групп 1 и 2
#  во временной интервал с 01.01.23 по 07.01.23 включительно)

q = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM simulator_20230120.feed_actions 
WHERE toDate(time) between '2023-01-01' and '2023-01-07'
    and exp_group in (1,2)
GROUP BY exp_group, user_id
"""

df_12 = pandahouse.read_clickhouse(q, connection=connection)

In [11]:
#T-test on groups 1(control) and 2(test)

t_test = stats.ttest_ind(df_12[df_12.exp_group == 1].ctr,
                df_12[df_12.exp_group == 2].ctr,
                equal_var=False)
print(f' p-value = {t_test.pvalue}')

 p-value = 0.4780623130874935


In [12]:
#Calculate global CTR
global_CTR_12 = sum(df_12[df_12.exp_group == 1].likes)/sum(df_12[df_12.exp_group == 1].views)

In [13]:
#Calculate linearized 'likes'
df_12['linearized_likes'] = df_12.apply(
    lambda x: x['likes'] - x['views']*global_CTR_12, axis=1)

In [14]:
t_test_12 = stats.ttest_ind(df_12[df_12.exp_group == 1].linearized_likes,
                df_12[df_12.exp_group == 2].linearized_likes,
                equal_var=False)
print(f' p-value = {t_test_12.pvalue}')

 p-value = 9.439432187037712e-10


T-tests on custom CTR and linearized CTR showed different results. 

Т-тесты на пользовательских CTR и линеаризованных CTR показали разные результаты
